In [1]:
import requests
import boto3
import json
import hashlib
from datetime import datetime, timezone
import logging
import os

In [2]:
# Filevine API & Auth Setup
FILEVINE_API_URL = ""
access_token = ""
user_id = ""
org_id = ""
session_id = ""

In [3]:
# AWS S3 Buckets
FILEVINE_BUCKET_NAME = ""
LOCAL_BUCKET_NAME = ""

In [4]:
# Local directory path (network mapped)
LOCAL_DIRECTORY_PATH = r""

In [5]:
# Logging
logging.basicConfig(
    filename='sync_log.txt',
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [6]:
def compute_md5_hash(input_str):
    md5 = hashlib.md5()
    md5.update(input_str.encode('utf-8'))
    return md5.hexdigest()

In [7]:
def refresh_access_token():
   
    logging.info("Refreshing access token.")

    # Generate the current timestamp in ISO 8601 format
    api_timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"

    # Given values (REPLACE with secure handling)
    api_key = ""
    api_secret = ""

    # Join values and compute hash
    data = "/".join([api_key, api_timestamp, api_secret])
    logging.info(f"Data to be hashed: {data}")

    hashed_data = compute_md5_hash(data)
    logging.info(f"Hashed Data: {hashed_data}")

    # Simulated token refresh (replace with actual call)
    return "new_access_token"

In [8]:
def download_file_from_url(url):
    try:
        response = requests.get(url)
        return response.content if response.status_code == 200 else None
    except Exception as e:
        logging.error(f"Error downloading: {e}")
        return None

In [9]:
def push_data_to_filevine(file_name, file_data, project_id, section_name="Docs"):
    url = f""
    headers = {
        "Authorization": f"Bearer {access_token}",
        "x-fv-userid": user_id,
        "x-fv-orgid": org_id,
        "x-fv-sessionid": session_id
    }

    files = {
        "file": (file_name, file_data),
        "section": (None, section_name)
    }

    try:
        response = requests.post(url, headers=headers, files=files)
        if response.status_code == 200:
            logging.info(f"Uploaded {file_name} to Filevine project {project_id} in section '{section_name}'")
        else:
            logging.error(f"Upload failed: {response.status_code}, {response.text}")
    except Exception as e:
        logging.error(f"Push to Filevine error: {e}")

In [10]:
def fetch_filevine_data(document_ids):
    global access_token
    headers = {
        "Authorization": f"Bearer {access_token}",
        "x-fv-userid": user_id,
        "x-fv-orgid": org_id,
        "x-fv-sessionid": session_id,
        "Content-Type": "application/json"
    }
    body = {"DocumentIds": document_ids}
    try:
        response = requests.get(FILEVINE_API_URL, headers=headers, json=body)
        if response.status_code == 200:
            documents = response.json().get("data", [])
            results = []
            for doc in documents:
                file_id = doc.get("id")
                download_url = doc.get("downloadUrl") or doc.get("fileUrl")
                if download_url:
                    content = download_file_from_url(download_url)
                    if content:
                        results.append((f"{file_id}.bin", content))
            return results
        elif response.status_code == 401:
            access_token = refresh_access_token()
            return fetch_filevine_data(document_ids)
    except Exception as e:
        logging.error(f"Filevine fetch error: {e}")
    return []

In [11]:
def save_to_s3(file_name, file_data, bucket_name, use_timestamp=True):
    s3 = boto3.client('s3', region_name='us-west-2')
    key = f"{datetime.now().strftime('%Y%m%d%H%M%S')}_{file_name}"
    try:
        s3.put_object(Bucket=bucket_name, Key=key, Body=file_data)
        logging.info(f"Uploaded {file_name} to {bucket_name}")
    except Exception as e:
        logging.error(f"S3 Upload Error: {e}")

In [12]:
def retrieve_from_s3(s3_key, bucket_name):
    s3 = boto3.client('s3', region_name='us-west-2')
    try:
        response = s3.get_object(Bucket=bucket_name, Key=s3_key)
        return response['Body'].read()
    except Exception as e:
        logging.error(f"S3 Read Error: {e}")
        return None

In [13]:
def list_s3_keys(bucket_name):
    s3 = boto3.client('s3', region_name='us-west-2')
    try:
        response = s3.list_objects_v2(Bucket=bucket_name)
        return [obj['Key'] for obj in response.get('Contents', [])]
    except Exception as e:
        logging.error(f"List S3 Error: {e}")
        return []

In [14]:
def save_to_local(data, filename):
    path = os.path.join(LOCAL_DIRECTORY_PATH, filename)
    try:
        with open(path, 'wb') as f:
            f.write(data)
        logging.info(f"Saved to local path: {path}")
    except Exception as e:
        logging.error(f"Local Save Error: {e}")

In [15]:
def sync_local_to_s3():
    for root, _, files in os.walk(LOCAL_DIRECTORY_PATH):
        for file in files:
            full_path = os.path.join(root, file)
            try:
                with open(full_path, 'rb') as f:
                    data = f.read()
                    save_to_s3(file, data, LOCAL_BUCKET_NAME)
            except Exception as e:
                logging.error(f"Local Read Error: {e}")

In [16]:
def sync_filevine_to_s3(document_ids):
    for file_name, file_data in fetch_filevine_data(document_ids):
        save_to_s3(file_name, file_data, FILEVINE_BUCKET_NAME)

In [17]:
def sync_s3_to_s3(key, source_bucket, dest_bucket):
    data = retrieve_from_s3(key, source_bucket)
    if data:
        save_to_s3(key, data, dest_bucket, use_timestamp=False)

In [18]:
def sync_s3_to_local(key):
    data = retrieve_from_s3(key, LOCAL_BUCKET_NAME)
    if data:
        save_to_local(data, os.path.basename(key))

In [19]:
def sync_s3_to_filevine(key, project_id):
    data = retrieve_from_s3(key, FILEVINE_BUCKET_NAME)
    if data:
        push_data_to_filevine(os.path.basename(key), data, project_id)

In [20]:
def full_filevine_sync_cycle(document_ids):
    sync_filevine_to_s3(document_ids)

    for key in list_s3_keys(FILEVINE_BUCKET_NAME):
        sync_s3_to_s3(key, FILEVINE_BUCKET_NAME, LOCAL_BUCKET_NAME)

    for key in list_s3_keys(LOCAL_BUCKET_NAME):
        sync_s3_to_local(key)

    sync_local_to_s3()

    for key in list_s3_keys(LOCAL_BUCKET_NAME):
        sync_s3_to_s3(key, LOCAL_BUCKET_NAME, FILEVINE_BUCKET_NAME)

    for key in list_s3_keys(FILEVINE_BUCKET_NAME):
        sync_s3_to_filevine(key)

In [ ]:
if __name__ == "__main__":
    document_ids = []  # Or change if needed — unrelated to projectId
    project_id =   #  Your actual Filevine project ID

    full_filevine_sync_cycle(document_ids)

    for key in list_s3_keys(FILEVINE_BUCKET_NAME):
        sync_s3_to_filevine(key, project_id)